# CAPSTONE PROJECT
## NEW BURGER RESTAURANT IN ANKARA


### INTRODUCTION
An entrepreneur, Deniz, wants to open a new burger restaurant in Ankara, the capital city of Turkey. 
Ankara's population is around 6 million and relatively young. It is famous for its universities and healthcare facilities. 
In order to help Deniz, we need to find the best location with the least competition and the highest number of possible costumers.

### DATA
In order to help Deniz, we need the following data in our analysis:
- Localities of Ankara from Wikipedia
- Long and lat of these localities from Open Street API

From Foursquare,
- Burger venues
- High school venues
- University venues
- Hospital vanues
- Office venues
of these localities will be retrieved.

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
#=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries and Installations Completed')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries and Installations Completed


In [3]:
website_url = requests.get("http://www.turkcewiki.org/wiki/Ankara%27n%C4%B1n_il%C3%A7eleri").text
soup = BeautifulSoup(website_url,'lxml')


In [4]:
My_table = soup.find('table',{'class':'wikitable sortable'})


links = My_table.findAll('a')
links

[<a href="#cite_note-19">[19]</a>,
 <a href="#cite_note-20">[20]</a>,
 <a href="#cite_note-21">[21]</a>,
 <a href="#cite_note-22">[22]</a>,
 <a href="#cite_note-23">[23]</a>,
 <a href="#cite_note-24">[24]</a>,
 <a href="#cite_note-25">[25]</a>,
 <a href="#cite_note-26">[26]</a>,
 <a href="#cite_note-27">[27]</a>,
 <a href="#cite_note-28">[28]</a>,
 <a href="#cite_note-29">[29]</a>,
 <a href="#cite_note-30">[30]</a>,
 <a href="#cite_note-31">[31]</a>,
 <a href="#cite_note-32">[32]</a>,
 <a href="#cite_note-33">[33]</a>,
 <a href="/wiki/2015_T%C3%BCrkiye_n%C3%BCfus_say%C4%B1m%C4%B1" title="2015 Türkiye nüfus sayımı">2015</a>,
 <a href="#cite_note-34">[34]</a>,
 <a href="/wiki/2016_T%C3%BCrkiye_n%C3%BCfus_say%C4%B1m%C4%B1" title="2016 Türkiye nüfus sayımı">2016</a>,
 <a href="#cite_note-35">[35]</a>,
 <a class="new" href="/w/index.php?title=2017_T%C3%BCrkiye_n%C3%BCfus_say%C4%B1m%C4%B1&amp;action=edit&amp;redlink=1" title="2017 Türkiye nüfus sayımı (sayfa mevcut değil)">2017</a>,
 <a href

In [5]:
Localities = []
for link in links:
    Localities.append(link.get('title'))
    

del Localities[0:24]



df = pd.DataFrame()
df['Localities'] = Localities
df['Localities'] = df['Localities'].str.split(',').str[0]
df

,Localities
0,Akyurt
1,Altındağ
2,Ayaş
3,Balâ
4,Beypazarı
5,Çamlıdere
6,Çankaya
7,Çubuk
8,Elmadağ
9,Etimesgut


In [6]:
def get_coords_local(locality, output_as='center'):

    url = '{0}{1}{2}'.format('http://nominatim.openstreetmap.org/search.php?q=',
                             locality+',Ankara,Turkey',
                             '&format=json&polygon=0')
    response = requests.get(url).json()[0]

    # parse response to list
    if output_as == 'boundingbox':
        lst = response[output_as]
        output = [float(i) for i in lst]
    if output_as == 'center':
        lst = [response.get(key) for key in ['lat','lon']]
        output = [float(i) for i in lst]
    return output
   

In [7]:

latitudeLst = []
longitudeLst = []

for index, row in df.iterrows():
    print(row[0])
    lat, long = get_coords_local(locality=row[0], output_as='center')
    latitudeLst.append(lat)
    longitudeLst.append(long)

df['Latitude'] = latitudeLst
df['Longitude'] = longitudeLst

df

Akyurt
Altındağ
Ayaş
Balâ
Beypazarı
Çamlıdere
Çankaya
Çubuk
Elmadağ
Etimesgut
Evren
Gölbaşı
Güdül
Haymana
Kalecik
Kahramankazan
Keçiören
Kızılcahamam
Mamak
Nallıhan
Polatlı
Pursaklar
Sincan
Şereflikoçhisar
Yenimahalle


,Localities,Latitude,Longitude
0,Akyurt,40.129780,33.083694
1,Altındağ,39.990974,33.000171
2,Ayaş,40.025591,32.238354
3,Balâ,39.555722,33.123549
4,Beypazarı,40.145747,31.917740
5,Çamlıdere,40.491480,32.475826
6,Çankaya,39.797276,32.945705
7,Çubuk,40.239254,33.028065
8,Elmadağ,39.917386,33.234236
9,Etimesgut,39.864584,32.569926


In [8]:

address = 'Ankara, Turkey'

geolocator = Nominatim(user_agent="capstoneProject")
location = geolocator.geocode(address, timeout=60, exactly_one=True)
latitude = location.latitude
longitude = location.longitude
print('The decimal coordinates of Ankara are {}, {}.'.format(latitude, longitude))

The decimal coordinates of Ankara are 39.9207774, 32.854067.


In [9]:
map_ankara= folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, local in zip(df['Latitude'], df['Longitude'], df['Localities']):
    label = '{}'.format(local)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_ankara)  
    
map_ankara

In [10]:
import urllib
import requests
limit = 500 # limit of number of venues returned by Foursquare API
radius = 5000 # define radius
CLIENT_ID = 'ZUVPBU2IBTYZLLVELWVP5OI4WI3XOP33SPTHB5PEZOAMDMTA'
CLIENT_SECRET = 'FY5QXXDNRDZQDUVXBWTREGF4ZRQGIZ1OOVUI2GTKOT2S3N5S'
VERSION = '20190915'
def getNearbyVenues(names, latitudes, longitudes, radius=5000, categoryIds=''):
    url=''
    response=''
    results=''
    nearby_venues=pd.DataFrame()
    try:
        venues_list=[]
        for name, lat, lng in zip(names, latitudes, longitudes):
            #print(name)

            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, limit)

            if (categoryIds != ''):
                url = url + '&categoryId={}'
                url = url.format(categoryIds)

            # make the GET request
            response = requests.get(url).json()
            results = response["response"]['venues']

            # return only relevant information for each nearby venue
            for v in results:
                success = False
                try:
                    category = v['categories'][0]['name']
                    success = True
                except:
                    pass

                if success:
                    venues_list.append([(
                        name, 
                        lat, 
                        lng, 
                        v['name'], 
                        v['location']['lat'], 
                        v['location']['lng'],
                        v['categories'][0]['name']
                    )])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Locality', 
                  'Locality Latitude', 
                  'Locality Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    except:
        print(url)
        print(response)
        print(results)
        print(nearby_venues)

    return(nearby_venues)

In [11]:
# BURGER RESTAURANTS
ankara_venues_burger = getNearbyVenues(names=df['Localities'], latitudes=df['Latitude'], longitudes=df['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d16c941735')
# HighSchools
ankara_venues_highschools = getNearbyVenues(names=df['Localities'], latitudes=df['Latitude'], longitudes=df['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d13d941735')

#Universities
ankara_venues_universities = getNearbyVenues(names=df['Localities'], latitudes=df['Latitude'], longitudes=df['Longitude'], radius=1000, categoryIds='4d4b7105d754a06372d81259')
#Business Centers
ankara_venues_business = getNearbyVenues(names=df['Localities'], latitudes=df['Latitude'], longitudes=df['Longitude'], radius=1000, categoryIds='56aa371be4b08b9a8d573517')


,Locality,Locality Latitude,Locality Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Akyurt,40.129780,33.083694,Vodafone Saka İletişim,40.130859,33.084984,Business Center
1,Akyurt,40.129780,33.083694,EnerjiSA Yetkili Ödeme Noktası Akyurt,40.131330,33.086383,Business Center
2,Akyurt,40.129780,33.083694,DNA Mühendislik Elektrik Proje,40.127380,33.075720,Business Center
3,Beypazarı,40.145747,31.917740,Tur Sürücü Kursu,40.147860,31.910244,Business Center
4,Beypazarı,40.145747,31.917740,Değişim Doğalgaz,40.151193,31.914152,Business Center


In [13]:
ankara_venues_burger.head()


,Locality,Locality Latitude,Locality Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Çubuk,40.239254,33.028065,Asuman's Mekan,40.235685,33.031674,Burger Joint
1,Elmadağ,39.917386,33.234236,burgerhastalığı king,39.921172,33.228766,Burger Joint
2,Kalecik,40.103903,33.412733,Istanbul Çiğköfte,40.097663,33.408827,Burger Joint
3,Keçiören,39.990795,32.851804,Hamburgerimm,39.992202,32.856607,Burger Joint
4,Keçiören,39.990795,32.851804,Hamburger,39.985447,32.858276,Burger Joint


In [14]:
ankara_venues_highschools.head()


,Locality,Locality Latitude,Locality Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Akyurt,40.129780,33.083694,Akyurt Anadolu Sağlık Meslek Lisesi,40.129450,33.086551,High School
1,Akyurt,40.129780,33.083694,Akyurt İmam Hatip Lisesi Ortaokulu,40.130658,33.086033,High School
2,Akyurt,40.129780,33.083694,akyurt imam hatip ortaokulu şantiyesi,40.133239,33.085869,High School
3,Balâ,39.555722,33.123549,Balâ Anadolu Lisesi,39.552923,33.124138,High School
4,Balâ,39.555722,33.123549,Kemal Şahin Anadolu Sağlık Meslek Lisesi,39.557112,33.118935,High School


In [15]:
ankara_venues_universities.head()


,Locality,Locality Latitude,Locality Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Akyurt,40.12978,33.083694,Akyurt Mesleki Ve Teknik Anadolu Lisesi,40.128480,33.086129,Trade School
1,Akyurt,40.12978,33.083694,Gazi Üniversitesi,40.128104,33.088521,College Academic Building
2,Akyurt,40.12978,33.083694,Gazi universitesi Ataturk MYO,40.130878,33.086705,University
3,Akyurt,40.12978,33.083694,"Özcan Spor Kulübü Body, Fitness.",40.129310,33.086147,College Gym
4,Akyurt,40.12978,33.083694,Mcd Mim Müs Müh İnşaat Ltd,40.127686,33.076237,College Administrative Building


In [16]:
ankara_venues_business.head()

,Locality,Locality Latitude,Locality Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Akyurt,40.129780,33.083694,Vodafone Saka İletişim,40.130859,33.084984,Business Center
1,Akyurt,40.129780,33.083694,EnerjiSA Yetkili Ödeme Noktası Akyurt,40.131330,33.086383,Business Center
2,Akyurt,40.129780,33.083694,DNA Mühendislik Elektrik Proje,40.127380,33.075720,Business Center
3,Beypazarı,40.145747,31.917740,Tur Sürücü Kursu,40.147860,31.910244,Business Center
4,Beypazarı,40.145747,31.917740,Değişim Doğalgaz,40.151193,31.914152,Business Center


In [17]:
#ADD them to the map
def addToMap(df, color, existingMap):
    for lat, lng, local, venue, venueCat in zip(df['Venue Latitude'], df['Venue Longitude'], df['Locality'], df['Venue'], df['Venue Category']):
        label = '{} ({}) - {}'.format(venue, venueCat, local)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)
        
map_ankara_burger = folium.Map(location=[latitude, longitude], zoom_start=12)
addToMap(ankara_venues_burger, 'yellow', map_ankara_burger)
map_ankara_burger



In [18]:
map_ankara_highschools = folium.Map(location=[latitude, longitude], zoom_start=12)
addToMap(ankara_venues_highschools, 'red', map_ankara_highschools)
map_ankara_highschools



In [19]:
map_ankara_universities = folium.Map(location=[latitude, longitude], zoom_start=12)
addToMap(ankara_venues_universities, 'green', map_ankara_universities)
map_ankara_universities



In [20]:
map_ankara_business = folium.Map(location=[latitude, longitude], zoom_start=12)
addToMap(ankara_venues_business, 'blue', map_ankara_business)
map_ankara_business

In [21]:
def addColumn(startDf, columnTitle, dataDf):
    grouped = dataDf.groupby('Locality').count()
    
    for n in startDf['Locality']:
        try:
            startDf.loc[startDf['Locality'] == n,columnTitle] = grouped.loc[n, 'Venue']
        except:
            startDf.loc[startDf['Locality'] == n,columnTitle] = 0




In [22]:
df.rename(columns={'Localities':'Locality'}, inplace=True)
addColumn(df, 'Burger', ankara_venues_burger)
addColumn(df, 'High Schools', ankara_venues_highschools)
addColumn(df, 'Universities', ankara_venues_universities)
addColumn(df, 'Offices', ankara_venues_business)
df

,Locality,Latitude,Longitude,Burger,High Schools,Universities,Offices
0,Akyurt,40.129780,33.083694,0.0,3.0,8.0,3.0
1,Altındağ,39.990974,33.000171,0.0,0.0,0.0,0.0
2,Ayaş,40.025591,32.238354,0.0,0.0,2.0,0.0
3,Balâ,39.555722,33.123549,0.0,3.0,13.0,0.0
4,Beypazarı,40.145747,31.917740,0.0,14.0,45.0,15.0
5,Çamlıdere,40.491480,32.475826,0.0,1.0,3.0,0.0
6,Çankaya,39.797276,32.945705,0.0,0.0,0.0,0.0
7,Çubuk,40.239254,33.028065,1.0,15.0,45.0,20.0
8,Elmadağ,39.917386,33.234236,1.0,7.0,44.0,8.0
9,Etimesgut,39.864584,32.569926,0.0,0.0,0.0,0.0


In [23]:
# Now we will weight each type of venue that might have neg or pos contribution to our decision
weight_burger = -1
weight_schools = 1
weight_uni = 2
weight_offices = 2

In [24]:
df_weighted = df[['Locality']].copy()
df_weighted['Score'] = df['Burger'] * weight_burger + df['High Schools'] * weight_schools + df['Universities'] * weight_uni + df['Offices'] * weight_offices
df_weighted = df_weighted.sort_values(by=['Score'], ascending=False)
df_weighted

,Locality,Score
20,Polatlı,194.0
22,Sincan,193.0
21,Pursaklar,152.0
7,Çubuk,144.0
24,Yenimahalle,143.0
16,Keçiören,142.0
4,Beypazarı,134.0
18,Mamak,112.0
8,Elmadağ,110.0
15,Kahramankazan,79.0


In [28]:
map_ankara_result = folium.Map(location=[latitude, longitude], zoom_start=15)

winner = df[df['Locality'] == 'Polatlı']

for lat, lng, local in zip(winner['Latitude'], winner['Longitude'], winner['Locality']):
    label = '{}'.format(local)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7).add_to(map_ankara_result) 

addToMap(ankara_venues_burger[ankara_venues_burger['Locality'] == 'Polatlı'], 'yellow', map_ankara_result)
addToMap(ankara_venues_highschools[ankara_venues_highschools['Locality'] == 'Polatlı'], 'red', map_ankara_result)
addToMap(ankara_venues_universities[ankara_venues_universities['Locality'] == 'Polatlı'], 'green', map_ankara_result)
addToMap(ankara_venues_business[ankara_venues_business['Locality'] == 'Polatlı'], 'blue', map_ankara_result)

map_ankara_result

## WINNER IS POLATLI